In [1]:
import pandas as pd
import numpy as np
# from tqdm import tqdm
# from tqdm import tnrange, tqdm_notebook
# from time import sleep
import difflib
import scipy
import operator
import difflib
from IPython.core.display import display,HTML
from tqdm import tnrange, tqdm_notebook

try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle



display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [2]:
path="/home/juliaponcela/at_NICO/Supply_networks/Results/"
filename="Simplified_stata_file_Pinf_vs_mulitiple_variables_monthly_slicing_dropped_overlaps_randomly_from_list.csv"

df= pd.read_csv(path +filename, sep=' ',low_memory=False)

print df.shape
df.head()

(282164, 64)


,P_inf,Mi,Cj,Mi_Cj,MCJ,num_trans,start_date_trans,end_date_trans,first_day_trans,last_day_trans,...,min_dist_j_to_inf,avg_dist_j_to_inf,degree_asym_ij,business_asym_ij,zs_error_rate_ij_year_degree,Dist,zip1,zip2,core_score_year_i,core_score_year_j
0,1,100055,2300055,100055_2300055,4,1,1987-05-01,1987-05-31,850,880,...,1,4.068182,0.404959,NaN,4.596074,0.343654,10018.0,10001,0.222222,0.333333
1,0,100055,400055,100055_400055,4,1,1987-05-01,1987-05-30,850,879,...,3,6.068182,0.111111,0.000000,-0.223313,0.838389,10018.0,10019,0.222222,0.333333
2,0,100286,1000019,100286_1000019,1,2,1985-03-01,1985-03-31,59,89,...,1,2.065934,0.990099,0.997706,-0.361477,NaN,NaN,NaN,1.000000,0.873899
3,1,100365,1000019,100365_1000019,1,4,1985-01-01,1985-12-31,0,364,...,1,2.348000,0.957224,0.954538,3.142229,NaN,10001.0,NaN,1.000000,0.873899
4,0,100365,1000019,100365_1000019,1,1,1986-01-01,1986-06-30,365,545,...,1,2.108108,0.978083,0.985517,-0.442647,NaN,10001.0,NaN,0.583333,0.835772


In [ ]:
# for c in df.columns:
#     print c

In [ ]:
# df=df.sort_values(by=['Mi_Cj', 'start_date_trans'], ascending=[1, 1])

#df['P_death_company'] = 0 # i add a new column  (it will indicate the last instance of a relationship)

In [ ]:
# df[df['Mi_Cj'] == "100365_1000019"]

In [ ]:
list_ids=df.Cj.unique()
len(list_ids)

In [3]:
dict_ID_activity={}
list_c=[]
list_m=[]
for row in df.iterrows():
    ID_m=row[1]["Mi"]
    ID_c=row[1]["Cj"]
    P_inf=row[1]["P_inf"]
    mi_cj=row[1]["Mi_Cj"]
    
    list_c.append(ID_c)
    list_m.append(ID_m)
    
    start_date=row[1]["start_date_trans"]
    end_date=row[1]["end_date_trans"]
    
    try:
        dict_ID_activity[ID_m]                    
    except KeyError:
        dict_ID_activity[ID_m]=[]
    
    
    try:
        dict_ID_activity[ID_c]                    
    except KeyError:
        dict_ID_activity[ID_c]=[]
    
    
    
    
    tupla=(start_date, end_date, P_inf, mi_cj)
    dict_ID_activity[ID_m].append(tupla)
    dict_ID_activity[ID_c].append(tupla)
    
    
print len(dict_ID_activity)   

8689


In [ ]:
#print len(set(list_c) & set(list_m)), len(set(list_c)), len(set(list_m))  # checking number of jobbers c, m

In [ ]:
df[((df['Mi'] == 100365) | (df['Cj'] == 100365)) & (df['P_inf'] == 1)]

In [5]:
name='aux_P_death_for_companies_old.csv'    ### OJO!!! now every transaction is duplicated!! from the Mi's point of view, and then from the Cj's!
f = open(path+name, 'wt')

print >> f, "ID start_date_trans end_date_trans P_inf Mi_Cj"

print len(dict_ID_activity)  
for ID in sorted(dict_ID_activity):
    lista=dict_ID_activity[ID]
    for item in lista:
     #   print ID, item[0], item[1], item[2],item[3], len(lista)
        print >> f, ID, item[0], item[1], item[2],item[3]
    #raw_input()

f.close()
print "written:", path+ name




path="/home/juliaponcela/at_NICO/Supply_networks/Results/"
filename='aux_P_death_for_companies_old.csv'

new_df= pd.read_csv(path +filename, sep=' ',low_memory=False)

print new_df.shape
new_df.head()      # old shape: 564328,5



new_df=new_df.sort_values(by=['ID', 'start_date_trans'], ascending=[1, 1])

new_df['P_death_company'] = (new_df.ID != new_df.ID.shift(-1)).astype(int)  # shifting is a way of checking whether the value of a cell in a given row is is the same as 1,2,3,... rows before/after


name='P_death_for_companies_old.csv'
new_df.to_csv(path+name, sep=';')
print "written:", path+name


new_df.tail(200)

8689
written: /home/juliaponcela/at_NICO/Supply_networks/Results/aux_P_death_for_companies_old.csv
(564328, 5)
written: /home/juliaponcela/at_NICO/Supply_networks/Results/P_death_for_companies_old.csv


,ID,start_date_trans,end_date_trans,P_inf,Mi_Cj,P_death_company
564149,20596612,2001-10-01,2001-10-02,1,20596612_412363,0
564150,20596612,2001-10-01,2001-10-30,0,20596612_412363,0
564188,20596612,2001-10-01,2001-10-30,0,20596612_5000055,0
564214,20596612,2001-10-01,2001-10-30,0,20596612_5009043,0
564107,20596612,2001-11-01,2001-11-30,1,20596612_12202696,0
564151,20596612,2001-11-01,2001-11-30,1,20596612_412363,0
564189,20596612,2001-11-01,2001-11-30,0,20596612_5000055,0
564215,20596612,2001-11-01,2001-11-30,0,20596612_5009043,0
564108,20596612,2001-12-01,2001-12-30,0,20596612_12202696,0
564152,20596612,2001-12-01,2001-12-31,1,20596612_412363,0


In [6]:
new_dict_of_dict={}
for ID in  dict_ID_activity:
    
    new_dict_of_dict[ID]={}
        
    for item in dict_ID_activity[ID]:
        
        tupla_dates=(item[0], item[1])
        p_inf=item[2]
        mi_cj= item[3]
        
        try:
            new_dict_of_dict[ID][tupla_dates]  # if the dates already exist
            
            if p_inf ==1:
                new_dict_of_dict[ID][tupla_dates]=(p_inf, mi_cj)   #if there is at least one error, then i set it to one
            
        except KeyError:
            new_dict_of_dict[ID][tupla_dates]=(p_inf, mi_cj) # if the dates didnt previously exist
    
print len(new_dict_of_dict)    

8689


In [7]:
name='aux_P_death_for_companies.csv'    ### OJO!!! now every transaction is duplicated!! from the Mi's point of view, and then from the Cj's!
f = open(path+name, 'wt')
print >> f, "ID start_date_trans end_date_trans P_inf Mi_Cj"




print len(new_dict_of_dict)  
for ID in sorted(new_dict_of_dict):
    dicc=new_dict_of_dict[ID]
    for llave in dicc:
        #print llave[0],llave[1], dicc[llave][0], dicc[llave][1]
        print >> f, ID, llave[0],llave[1], dicc[llave][0], dicc[llave][1]
        #print >> f, ID, item[0], item[1], item[2],item[3]
    #print "lenght old dict:", len(dict_ID_activity[ID]), "   new lenght:", len(dicc)
    #raw_input()

f.close()
print "written:", path+ name







path="/home/juliaponcela/at_NICO/Supply_networks/Results/"
filename='aux_P_death_for_companies.csv'
new_df_no_overlap= pd.read_csv(path +filename, sep=' ',low_memory=False)

print new_df_no_overlap.shape  # old shape: 564328,5    new, without overlap from firm-centered perspective: (230309, 5)
#new_df_no_overlap.head()      








new_df_no_overlap=new_df_no_overlap.sort_values(by=['ID', 'start_date_trans'], ascending=[1, 1])
new_df_no_overlap['P_death_company'] = (new_df_no_overlap.ID != new_df_no_overlap.ID.shift(-1)).astype(int)  # shifting is a way of checking whether the value of a cell in a given row is is the same as 1,2,3,... rows before/after


name='P_death_for_companies.csv'
new_df_no_overlap.to_csv(path+name, sep=';')
print "written:", path+name


new_df_no_overlap.tail(200)

8689
written: /home/juliaponcela/at_NICO/Supply_networks/Results/aux_P_death_for_companies.csv
(230309, 5)
written: /home/juliaponcela/at_NICO/Supply_networks/Results/P_death_for_companies.csv


,ID,start_date_trans,end_date_trans,P_inf,Mi_Cj,P_death_company
230117,20596582,1998-05-01,1998-05-31,0,100948_20596582,0
230130,20596582,1998-06-01,1998-06-30,0,100948_20596582,0
230127,20596582,1998-07-01,1998-07-31,0,100948_20596582,0
230115,20596582,1998-08-01,1998-08-31,0,100948_20596582,0
230124,20596582,1998-09-01,1998-09-30,0,100948_20596582,0
230128,20596582,1998-10-01,1998-10-31,0,100948_20596582,0
230111,20596582,1998-11-01,1998-11-30,0,100948_20596582,0
230133,20596582,1998-12-01,1998-12-31,0,100948_20596582,0
230132,20596582,1999-01-01,1999-01-31,0,100948_20596582,0
230123,20596582,1999-02-01,1999-02-28,0,100948_20596582,0


In [ ]:
# for mi_cj in list_ids:
#     selection = df[df['Mi_Cj'] == mi_cj]
#     #print mi_cj, selection.shape
#     #raw_input()

In [ ]:
# def add_Pdeath(input_df):
    
   
#    # print input_df#.shape,len(input_df)
#     #print "index",input_df.index[-1]
    
#     index_last= input_df.index[-1]    #### Int64Index([37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], dtype='int64')    
#     input_df.set_value(index_last, 'P_death', 1)
        
        
        
#     #print input_df
# #     print 
# #     print 
# #     raw_input()
#     return input_df


# new_df = df.groupby('Mi_Cj').apply(add_Pdeath)

#grouped= grouped.reset_index()  # to reset the indexes (instead of using the authour_id as index, i create a new one)



# df['P_death_company'] = (df.Mi_Cj != df.Mi_Cj.shift(-1)).astype(int)   # shifting is a way of checking whether the value of a cell in a given row is is the same as 1,2,3,... rows before/after

In [ ]:
# a faster way of doing the same:

# df[['Mi_Cj',"P_death_company","P_inf"]].tail(1000)

In [ ]:

# writer = pd.ExcelWriter(path+filename.strip(".csv")+"_Pdeath_company.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})

# # Convert the dataframe to an XlsxWriter Excel object.
# df.to_excel(writer, sheet_name='Sheet1')
# # Close the Pandas Excel writer and output the Excel file.
# writer.save()

# print "xlsx done:",path+filename.strip(".csv")+"_Pdeath.xlsx"





In [ ]:

# df.to_csv(path+filename.strip(".csv")+"_Pdeath.tsv", sep='\t', encoding='utf-8')#, columns = list_headers)
# print "csv done:", path+filename.strip(".csv")+"_Pdeath.tsv"